In [3]:
import keras
from keras.datasets import mnist
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

## Podešavanje osnovnih parametara i ,,sređivanje dataseta-a" 

In [4]:
num_classes = 10
epochs = 15
batch_size = 64

img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)


In [5]:
print(x_train[0].shape)

(28, 28, 1)


## Podešavanje arhitekture konvolucione mreže 

In [6]:
classifier = Sequential()

classifier.add(Conv2D(64, (2, 2), input_shape = (28, 28, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (2, 2), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (1, 1)))
classifier.add(Dropout(0.2))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 10, activation = 'softmax'))

classifier.compile(optimizer = keras.optimizers.Adadelta(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Treniranje i rezultati konvolucione mreže 

In [7]:
classifier.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_data = (x_test, y_test))
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 77s 1ms/step - loss: 0.2118 - acc: 0.9352 - val_loss: 0.0591 - val_acc: 0.9805
Epoch 2/15
60000/60000 [==============================] - 75s 1ms/step - loss: 0.0870 - acc: 0.9742 - val_loss: 0.0463 - val_acc: 0.9841
Epoch 3/15
60000/60000 [==============================] - 75s 1ms/step - loss: 0.0668 - acc: 0.9803 - val_loss: 0.0388 - val_acc: 0.9868
Epoch 4/15
60000/60000 [==============================] - 75s 1ms/step - loss: 0.0600 - acc: 0.9817 - val_loss: 0.0391 - val_acc: 0.9865
Epoch 5/15
60000/60000 [==============================] - 75s 1ms/step - loss: 0.0528 - acc: 0.9843 - val_loss: 0.0375 - val_acc: 0.9879
Epoch 6/15
60000/60000 [==============================] - 76s 1ms/step - loss: 0.0529 - acc: 0.9842 - val_loss: 0.0355 - val_acc: 0.9875
Epoch 7/15
60000/60000 [==============================] - 76s 1ms/step - loss: 0

In [8]:
classifier.save('model_1.h5')